In [ ]:
!pip install langchain

In [ ]:
!pip install pinecone

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install pinecone-client

In [ ]:
import re
import pandas as pd
from io import BytesIO
from typing import Tuple, List
import pickle
import csv
import os
import json


import pinecone
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings


from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
product_data = pd.read_csv('/content/drive/MyDrive/products.csv')

In [ ]:
product_data.head(2)

,Name,Variant Name,Price,Size,Material,Compatibility,Features,URL,Meta Title,Meta Description,Product Description
0,iPhone 7 Series - Hybrid Armor,iPhone 7 Plus / Black / In Stock,39.99,"['Dimensions: 6.46 x 3.28 x 0.44 inches', 'Pro...","['TPU: Thermoplastic Polyurethane', 'PC: Polyc...",['Apple - iPhone 7 Plus'],['New generation of dual-layered protection fo...,https://www.spigen.com/products/iphone-7-serie...,iPhone 7 Series Hybrid Armor Case - Spigen.com...,Protect your iPhone 7 Plus with one of our tou...,\n\n\n\n\n\nMore than just protection.\nExplor...
1,Tesla Model 3 - Armrest Console Organizer | TO...,Black / In Stock,39.99,['Package Weight: 12.5 oz'],['PC: Polycarbonate'],['Tesla - Model 3 (2024) Highland'],"['Made from PC for durability', 'Precise fit i...",https://www.spigen.com/products/tesla-model-3-...,Tesla Model 3 Armrest Console Organizer - Spig...,Keep your Tesla Model 3 organized. Shop our Ar...,\n\n\n\n\n\nMore than just protection.\nExplor...


In [ ]:
# converting csv to json

csv_file_path = '/content/drive/MyDrive/products.csv'
json_file_path = '/content/drive/MyDrive/products_data.json'

if not os.path.exists(json_file_path):
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        data = []

        for row in reader:
            data_object = {
                "Name": row["Name"],
                "Variant Name": row["Variant Name"],
                "Price": row["Price"],
                "Size": row["Size"],
                "Material": row["Material"],
                "Compatibility": row["Compatibility"],
                "Features": row["Features"],
                "URL": row["URL"],
                "Meta Title": row["Meta Title"],
                "Meta Description": row["Meta Description"],
                "Product Description": row["Product Description"]
            }

            data.append(data_object)

    json_data = json.dumps(data, indent=4)

    with open(json_file_path, 'w') as jsonfile:
        jsonfile.write(json_data)
else:
    print("JSON file already exists.")

JSON file already exists.


In [ ]:
# Splitting json file

def split_json_to_files(json_file_path, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(json_file_path, 'r') as json_file:
        products = json.load(json_file)


    for i, product in enumerate(products):
        product_details = {
            'id': i,
            "Name": product["Name"],
            "Variant Name": product["Variant Name"],
            "Price": product["Price"],
            "Size": product["Size"],
            "Material": product["Material"],
            "Compatibility": product["Compatibility"],
            "Features": product["Features"],
            "URL": product["URL"],
            "Meta Title": product["Meta Title"],
            "Meta Description": product["Meta Description"],
            "Product Description": product["Product Description"]
        }

        output_file_path = os.path.join(output_dir, f'{i}.json')
        with open(output_file_path, 'w') as output_file:
            json.dump(product_details, output_file, indent=4)

        #print(f"Saved details of product {i} to {output_file_path}")


In [ ]:
output_dir = '/content/drive/MyDrive/Prod_one_each'
json_file_path = '/content/drive/MyDrive/products_data.json'
split_json_to_files(json_file_path, output_dir)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Get embeddings

def embed_product_information(json_file_path):
    with open(json_file_path, 'r') as json_file:
        product_data = json.load(json_file)

    text_attributes = [
        product_data['Name'],
        product_data['Variant Name'],
        product_data['Price'],
        ', '.join(eval(product_data['Size'])) if product_data['Size'] != 'NA' else '',
        ', '.join(eval(product_data['Material'])) if product_data['Material'] != 'NA' else '',
        ', '.join(eval(product_data['Compatibility'])) if product_data['Compatibility'] != 'NA' else '',
        ', '.join(eval(product_data['Features'])) if product_data['Features'] != 'NA' else '',
            product_data['Meta Title'],
        product_data['Meta Description'],
        product_data['Product Description']
    ]
    concatenated_text = '\n'.join(text_attributes)
    concatenated_text_with_image_url = concatenated_text + '\n' + product_data['URL']

    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    embeddings_model = SentenceTransformer(model_name)
    embedding = embeddings_model.encode(concatenated_text_with_image_url )

    return embedding


In [ ]:
# # Example usage
# json_file_path = '/content/drive/MyDrive/Prod_one_each/0.json'  # Path to your JSON file
# #embedding = embed_product_information(json_file_path)
# #print("Embedding:", embedding)
# embed = embed_product_information(json_file_path)
# print(embed)

In [ ]:
def process_json_files(directory):
    embeddings_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            json_file_path = os.path.join(directory, filename)
            embedding = embed_product_information(json_file_path)

            with open(json_file_path, 'r') as json_file:
              product_data = json.load(json_file)

            product_id = product_data['id']
            product_name = product_data['Name']
            embeddings_data.append({'id': product_id, 'name': product_name, 'embedding': embedding.tolist()})

            # Save embedding data to a separate JSON file for each product
            output_json_path = os.path.join('/content/drive/MyDrive/embeddings', f'embedding_{product_id}.json')
            with open(output_json_path, 'w') as output_json_file:
                json.dump({'id': product_id, 'name': product_name, 'embedding': embedding.tolist()}, output_json_file)

    return embeddings_data



In [ ]:
# Directory containing JSON files
directory_path = '/content/drive/MyDrive/Prod_one_each'

# Process JSON files in the directory
embeddings_data = process_json_files(directory_path)

# Save embeddings data to a JSON file
# with open('all_embeddings.json', 'w') as all_embeddings_file:
#     json.dump(embeddings_data, all_embeddings_file)

In [ ]:
with open('all_embeddings.json', 'w') as all_embeddings_file:
    json.dump(embeddings_data, all_embeddings_file)

In [ ]:
json_embed_path = "all_embeddings.json"

with open(json_embed_path, 'r') as json_file:
    embedded_product_data = json.load(json_file)

# Find the maximum length of a single embedding
max_embedding_length = max(len(embed["embedding"]) for embed in embedded_product_data)

print("Maximum length of a single embedding:", max_embedding_length)


Maximum length of a single embedding: 384


In [ ]:
#Connect with pinecone

pc = Pinecone(
        api_key='0ea3ebe6-7fc2-4312-9033-2f979891fd7e'
    )

if 'productdb' not in pc.list_indexes().names():
        pc.create_index(
            name='productdb',
            dimension= 384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

In [ ]:
from pinecone import Index

In [ ]:
#upserting into the database
# Set your index name (create it in Pinecone beforehand)
index_name = "productdb"


def store_product_embeddings(embeddings_dir):
    index = Index(api_key='0ea3ebe6-7fc2-4312-9033-2f979891fd7e', host ='https://productdb-hwxvhuf.svc.aped-4627-b74a.pinecone.io')

    for filename in os.listdir(embeddings_dir):
        if not filename.endswith(".json"):
            continue  # Ignore non-JSON files

        # Load product information
        with open(os.path.join(embeddings_dir, filename), "r") as f:
            product_data = json.load(f)

        # Extract product ID, name, and embedding from JSON data
        product_id = product_data.get("id")  # Use get() for safer access
        #product_name = product_data.get("name")  # Use get() for safer access
        embedding = product_data.get("embedding")  # Use get() for safer access

        if not all([product_id, embedding]):  # Check for missing data
            print(f"Warning: Skipping {filename} due to missing data.")
            continue

        # Combine product information and embedding into a single dictionary
        vector_data = {

                "id": str(product_id),  # Use product ID as embedding ID (optional)
                "values": embedding,
        }

        # Upsert data into Pinecone (using product ID as primary identifier)
        index.upsert([vector_data], index_name)

    print("Product information and embeddings stored successfully!")


In [ ]:
# Example usage
embeddings_dir = "/content/drive/MyDrive/embeddings"  # Update with your path
store_product_embeddings(embeddings_dir)

Product information and embeddings stored successfully!


In [ ]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embeddings_model = SentenceTransformer(model_name)

In [ ]:
query = "apple phone cover"
query_embeddings = embeddings_model.encode(query)
query_embeddings = query_embeddings.tolist()
print(query_embeddings)

[-0.11660336703062057, 0.1270860880613327, 0.12138770520687103, 0.030675793066620827, 0.07250659167766571, 0.004728056024760008, 0.0878557488322258, 0.06143239140510559, 0.038871943950653076, 0.005005570128560066, 0.0003883018798660487, 0.028040332719683647, 0.021794481202960014, -0.01434045098721981, 0.010852674953639507, -0.06293729692697525, -0.03630441799759865, -0.09061869233846664, -0.06224619969725609, 0.01490782294422388, -0.04189540445804596, 0.024043532088398933, 0.04683830589056015, -0.05436096712946892, -0.031049687415361404, 0.03940298780798912, 0.02017873525619507, 0.004445942584425211, -0.053172796964645386, -0.03020775504410267, -0.018874283879995346, 0.03221961110830307, 0.07780657708644867, 0.12149078398942947, -0.012587441131472588, -0.03978925570845604, -0.029575539752840996, 0.0984557718038559, -0.052122563123703, 0.05894490331411362, -0.05219600349664688, 0.030880417674779892, -0.008696911856532097, 0.06430859118700027, 0.005726136267185211, 0.03567099571228027, -

In [ ]:
index = Index(api_key='0ea3ebe6-7fc2-4312-9033-2f979891fd7e', host ='https://productdb-hwxvhuf.svc.aped-4627-b74a.pinecone.io')


In [ ]:
!pip install --upgrade pinecone

In [ ]:
results = index.query(
    index_name='productdb',
    vector= query_embeddings,
    top_k = 10
)

In [ ]:
print(results)

{'matches': [], 'namespace': '', 'usage': {'read_units': 0}}


In [ ]:
results= index.query(
    namespace="productdb",
    vector= query_embeddings,
    top_k=10,
    include_values=True,
    include_metadata=True,
    #filter={"genre": {"$eq": "action"}}
)

In [ ]:
for result in results["matches"]:
    product_id = result["id"]  # Assuming "id" field exists in embedding
    similarity_score = result["score"]
    print(f"Product ID: {product_id}, Similarity Score: {similarity_score}")

Product ID: 1424, Similarity Score: 0.623001516
Product ID: 89, Similarity Score: 0.605100095
Product ID: 456, Similarity Score: 0.603256345
Product ID: 457, Similarity Score: 0.601627409
Product ID: 458, Similarity Score: 0.600869656
Product ID: 459, Similarity Score: 0.600731313
Product ID: 2548, Similarity Score: 0.600025237
Product ID: 1425, Similarity Score: 0.599646211
Product ID: 2549, Similarity Score: 0.59573251
Product ID: 2547, Similarity Score: 0.593858957
